In [3]:
import json
import os
import re
import copy
from tqdm import tqdm

In [4]:

# json_data['images']
# json_data['images']['file_name']
# json_data['images']['coco_url']
# json_data['images']['height']
# json_data['images']['width']
# json_data['images']['flickr_url']
# json_data['images']['id']
# -----------------------
# @@@@@@@@@@@@@@@@@@@@@@@
# -----------------------
# json_data['annotations']
# json_data['annotations']['bbox']
# json_data['annotations']['image_id']


# {path}/yolov3-master/coco/images/{file_name}/.jpg   --->   {path}/yolov3-master/data/coco1cls.txt
# {path}/yolov3-master/coco/labels/{file_name}/.txt   --->   0 0.xx 0.xx 0.xx 0.xx      (class, x_center, y_center, width, height)
#                                                            0 0.xx 0.xx 0.xx 0.xx ...


In [23]:
class Error(Exception):
    pass

In [56]:
class Download:
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.current_path = os.getcwd()
        self.file_names, self.route = self.get_file_name()
        self.cls_path = 'coco1cls.txt'
        
    def read_json(self,file_path):
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)
        
        return json_data
    
    def get_file_name(self):
        tmp = []
        route = []
        for i,x in enumerate(self.file_paths):
            
            name_i = ''
            image_path = copy.deepcopy(self.current_path)
            image_path = image_path.replace('yolov3-master','')
            image_path += 'cocodataset/images/'

            full_name = os.path.basename(x)
            name = re.findall('(.*)\.', full_name)
            
            if '2017' in name[0]:
                if 'train' in name[0]:
                    name_i = 'train2017'
                elif 'val' in name[0]:
                    name_i = 'val2017'
                    
            elif '2014' in name[0]:
                if 'train' in name[0]:
                    name_i = 'train2014'
                elif 'val' in name[0]:
                    name_i = 'val2014'
            
            image_path = image_path + name_i + '_person/'
                    
            route.append(image_path) 
            tmp += name
            
        return tmp, route
    
    def get_bbox(self, image, anno):
        
        # find center
        # normalize
        bbox = []
        x_center = anno['bbox'][0] + anno['bbox'][2]/2
        y_center = anno['bbox'][1] + anno['bbox'][3]/2
        
        height, width = image['height'], image['width']
        
        bbox.append(round(x_center/width,6))
        bbox.append(round(y_center/height,6))
        bbox.append(round(anno['bbox'][2]/width,6))
        bbox.append(round(anno['bbox'][3]/height,6))
        
        return bbox # normalized list: [x,y,w,h]
        
    def __call__(self):
        # 레이블을 저장하고
        # coco1cls.txt에 경로를 저장해야함
        for i,x in enumerate(self.file_paths):
            json_data = self.read_json(x)
            
            if 'val' in self.file_names[i]:
                self.cls_path = 'coco1cls_val.txt'
                
            memory = {}
            for j,d in enumerate(tqdm(json_data['images'])):
                route = self.route[i] + d['file_name']
                
                label_route = copy.deepcopy(route)
                label_route = label_route.replace('images','labels')
                label_route = label_route[:-3]+'txt'
                
                try:
                    if os.path.exists(route) == False:
                        raise Error()
                except:
                    continue
                    
                # write coco1cls.txt, coco1cls_val.txt
                f = open(self.current_path+'/data/{}'.format(self.cls_path),'a')
                f.write(route+'\n')
                f.close()
                        
                # save label
                for k,a in enumerate(json_data['annotations']):
                    try:
                        if memory[k] == 1:
                            continue
                    except:
                        if a['image_id'] == d['id']:
                            f = open(label_route, 'a')
                            bbox = self.get_bbox(d,a)
                            line = '0 {0} {1} {2} {3}\n'.format(bbox[0],bbox[1],bbox[2],bbox[3])
                            f.write(line)
                            f.close()
                        
                            memory[k] = 1

In [57]:
ann_path = '../cocodataset/annotations/'
train_file_list = ['../cocodataset/annotations/instances_train2014_person.json', 
                   '../cocodataset/annotations/instances_train2017_person.json',
                   '../cocodataset/annotations/instances_val2014_person.json', 
                   '../cocodataset/annotations/instances_val2017_person.json']

train_2017_list = ['../cocodataset/annotations/instances_train2017_person.json',
                   '../cocodataset/annotations/instances_val2017_person.json']
downloader = Download(train_file_list)
downloader()
# python3 train.py --data data/coco_1cls.data

100%|██████████| 2693/2693 [00:05<00:00, 455.03it/s]


In [51]:
# 2014 개수 확인해보기
# 2017 저장 안된 이유 확인하기

f=open('./data/coco1cls.txt')

In [ ]:
# !python3 train.py --cfg yolov3-spp-1cls.cfg --data data/coco_1cls.data